In [ ]:
#### References

https://www.seleniumhq.org/docs/03_webdriver.jsp#chapter03-reference
https://sites.google.com/a/chromium.org/chromedriver/downloads
https://pypi.org/project/selenium/
https://selenium-python.readthedocs.io/getting-started.html#simple-usage


In [1]:
# A basic selenium based program
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

In [1]:
from selenium import webdriver

In [2]:
# Create a new instance of the Firefox driver
driver = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: Chrome version must be between 70 and 73
  (Driver info: chromedriver=73.0.3683.68 (47787ec04b6e38e22703e856e101e840b65afe72),platform=Windows NT 6.1.7601 SP1 x86_64)


In [3]:
from selenium import webdriver

browser = webdriver.Firefox()
browser.get('http://seleniumhq.org/')

WebDriverException: Message: connection refused


In [6]:
# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

WebDriverException: Message: connection refused


In [ ]:
# go to the google home page
driver.get("http://www.google.com")

# the page is ajaxy so the title is originally this:
print driver.title

# find the element that's name attribute is q (the google search box)
inputElement = driver.find_element_by_name("q")

# type in the search
inputElement.send_keys("cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()

try:
    # we have to wait for the page to refresh, the last thing that seems to be updated is the title
    WebDriverWait(driver, 10).until(EC.title_contains("cheese!"))

    # You should see "cheese! - Google Search"
    print driver.title

finally:
    driver.quit()

In [4]:
# URL and Request code for BeautifulSoup

url_filter_bc = 'https://www.backcountry.com/msr-miniworks-ex-ceramic-water-filter?skid=CAS0479-CE-ONSI&ti=U2VhcmNoIFJlc3VsdHM6bXNyOjE6MTE6bXNy'
res_filter_bc = requests.get(url_filter_bc, headers = {'User-agent' : 'notbot'})


# Function that scrapes the reivews

def scrape_bc(request, website):
    newlist = []
    soup = BeautifulSoup(request.content, 'lxml')
    newsoup = soup.find('div', {'id': 'the-wall'})
    reviews = newsoup.find('section', {'id': 'wall-content'})

    for row in reviews.find_all('section', {'class': 'upc-single user-content-review review'}):
        newdict = {}
        newdict['review']  = row.find('p', {'class': 'user-content__body description'}).text
        newdict['title']   = row.find('h3', {'class': 'user-content__title upc-title'}).text
        newdict['website'] = website

        newlist.append(newdict)

    df = pd.DataFrame(newlist)
    return df


# function that uses Selenium and combines that with the scraper function to output a pandas Dataframe

def full_bc(url, website):
    driver = connect_to_page(url, headless=False)
    request = requests.get(url, headers = {'User-agent' : 'notbot'})
    time.sleep(5)
    full_df = pd.DataFrame()
    while True:
        try:
            loadMoreButton = driver.find_element_by_xpath("//a[@class='btn js-load-more-btn btn-secondary pdp-wall__load-more-btn']")
            time.sleep(2)
            loadMoreButton.click()
            time.sleep(2)
        except:
            print('Done Loading More')

#             full_json = driver.page_source
            temp_df = pd.DataFrame()
            temp_df = scrape_bc(request, website)

            full_df = pd.concat([full_df, temp_df], ignore_index = True)

            time.sleep(7)
            driver.quit()
            break

    return  full_df 